Входная коллекция данных состоит из двух частей:
1. заголовки новостных сообщений, входящих в топ-25 по популярности, опубликованных на  Reddit WorldNews Channel
2. бинаризованный промышленный индекса Доу — Джонса DJIA (0, если значение индекса за день понизилось, 1 – если осталось на прежнем уровне или выросло). 

Данные собраны за период с 2008-08-08 до 2016-07-01.

Данные представлены в файле Combined_News_DJIA.csv. Первая колонка – дата, вторая – бинаризованный промышленный индекса Доу — Джонса, оставшиеся 25 – заголовки новостей. 

### Часть 1 [3 балла] Предварительная обработка текстов
Проведите предобработку текстов: если считаете нужным, выполните токенизацию, приведение к нижнему регистру, лемматизацию и/или стемминг. 
Ответьте на следующие вопросы:
1. Есть ли корреляция между средней длинной текста за день и DJIA?
2. Есть ли корреляция между количеством упоминаний Барака Обамы и США в день и DJIA? Учтите разные варианты написания США.
3. Каких статей больше: статей о России и Путине или об Исламском государстве (запрещенной законом РФ террористическая организации)?
4. О каких кризисах (crisis) пишут статьи?


In [17]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import re
import numpy as np
import pymorphy2 as pm

In [155]:
df = pd.read_csv('Combined_News_DJIA.csv')

In [164]:
morph = pm.MorphAnalyzer()
def set_clean(s):
    try:
        # print(type(s))
        clean_line = re.sub('[\W\d_-]+', ' ', s.lower().strip())
        return re.sub(' +', ' ', clean_line)
    except AttributeError:
        ''#print ("this was a series")
print(set_clean((df['Top1'][1])))

b why wont america and nato help us if they wont help us now why did we help them in iraq 


In [161]:
df_small = df.drop(['Date', 'Label'], axis=1) # textx only

In [165]:
df_small.applymap(set_clean)

,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,b georgia downs two russian warplanes as count...,b breaking musharraf to be impeached,b russia today columns of troops roll into sou...,b russian tanks are moving towards the capital...,b afghan children raped with impunity u n offi...,b russian tanks have entered south ossetia whi...,b breaking georgia invades south ossetia russi...,b the enemy combatent trials are nothing but a...,b georgian troops retreat from s osettain capi...,b did the u s prep georgia for war with russia,...,b georgia invades south ossetia if russia gets...,b al qaeda faces islamist backlash,b condoleezza rice the us would not act to pre...,b this is a busy day the european union has ap...,b georgia will withdraw soldiers from iraq to ...,b why the pentagon thinks attacking iran is a ...,b caucasus in crisis georgia invades south oss...,b indian shoe manufactory and again in a serie...,b visitors suffering from mental illnesses ban...,b no help for mexico s kidnapping surge
1,b why wont america and nato help us if they wo...,b bush puts foot down on georgian conflict,b jewish georgian minister thanks to israeli t...,b georgian army flees in disarray as russians ...,b olympic opening ceremony fireworks faked,b what were the mossad with fraudulent new zea...,b russia angered by israeli military sale to g...,b an american citizen living in s ossetia blam...,b welcome to world war iv now in high definition,b georgia s move a mistake of monumental propo...,...,b israel and the us behind the georgian aggres...,b do not believe tv neither russian nor georgi...,b riots are still going on in montreal canada ...,b china to overtake us as largest manufacturer,b war in south ossetia pics,b israeli physicians group condemns state tort...,b russia has just beaten the united states ove...,b perhaps the question about the georgia russi...,b russia is so much better at war,b so this is what it s come to trading sex for...
2,b remember that adorable year old who sang at ...,b russia ends georgia operation,b if we had no sexual harassment we would have...,b al qa eda is losing support in iraq because ...,b ceasefire in georgia putin outmaneuvers the ...,b why microsoft and intel tried to kill the xo...,b stratfor the russo georgian war and the bala...,b i m trying to get a sense of this whole geor...,b the us military was surprised by the timing ...,b u s beats war drum as iran dumps the dollar,...,b u s troops still in georgia did you know the...,b why russias response to georgia was right,b gorbachev accuses u s of making a serious bl...,b russia georgia and nato cold war two,b remember that adorable year old who led your...,b war in georgia the israeli connection,b all signs point to the us encouraging georgi...,b christopher king argues that the us and nato...,b america the new mexico,b bbc news asia pacific extinction by man not ...
3,b u s refuses israel weapons to attack iran re...,b when the president ordered to attack tskhinv...,b israel clears troops who killed reuters came...,b britain s policy of being tough on drugs is ...,b body of year old found in trunk latest ranso...,b china has moved million quake survivors into...,b bush announces operation get all up in russi...,b russian forces sink georgian ships,b the commander of a navy air reconnaissance s...,b of cnn readers russia s actions in georgia j...,...,b elephants extinct by,b us humanitarian missions soon in georgia if ...,b georgia s ddos came from us sources,b russian convoy heads into georgia violating ...,b israeli defence minister us against strike o...,b gorbachev we had no choice,b witness russian forces head towards tbilisi ...,b quarter of russians blame u s for conflict p...,b georgian president says us military will tak...,b nobel laureate aleksander solzhenitsyn accus...
4,b all the experts admit that we should legalis...,b war in south osetia pictures made by a russi...,b 

Считаем среднюю длину текста за день и ее отношение к индексу

In [22]:
df_clean = pd.DataFrame(df)

In [23]:
def meaning(columns, index):
    den = len(columns)
    acc = 0
    for i in range(den):
        try:
            acc+=len(columns[i][index])
        except TypeError:
            den-=1
    return acc/den
def meanings(columns):
    l = len(columns[0])
    return [{'dj': df['Label'][i], 'meaning': meaning(columns, i)} for i in range(l)]
cols = [df["Top{0}".format(i)] for i in range(1, 26)]
cols_new = meanings(cols)

In [24]:
type(df_clean)

pandas.core.frame.DataFrame

In [25]:
cols_df = pd.DataFrame(cols_new)


In [142]:
cols_sort = cols_df.sort_values(by='meaning', ascending=False)

Упоминания Обамы, США и индекс??

Кого чаще упоминают: Россию или ИГИЛ?

In [138]:
def frequensy(ptrn):
    def ret(columns):
        с = 0
        for col in columns:
            for v in col:
                if (type(v)==str and re.search(ptrn, v)):
                    с += 1
        return print(с)
    return ret
frequensy('Russia')(cols)
frequensy('Russian')(cols) # 923
frequensy('Putin')(cols) # 371


2736
1232
528


In [298]:
frequensy('Russia')(cols)
frequensy('ISIS')(cols) # 389
frequensy('IS')(cols) # 858

2736
580
894


О каких кризисах пишут в статьях?

In [301]:
def crisis(columns):
    crisis_list = set()
    for col in columns:
        for v in col:
            if (type(v)==str and re.search('crisis', v)): # иногда вместо str в dataFrame float
                pattern = "((\\w+\\W+){,2}\\b|\w+)crisis" # считываем предыдущие кризису два слова,
                                                          # чтобы узнать, о каком кризисе идёт речь
                try:
                    vv = re.search(pattern, v).group(0)
                    crisis_list.add(vv)
                except AttributeError:
                    print("\n%%%%%  ")
                    print(v)
                    print(re.search(pattern, v))
    return crisis_list
r = crisis(cols)

### Часть 2 [5 баллов] Классификация

Вам предстоит решить следующую задачу: по текстам новостей за день определить, вырастет или понизится DJIA. То есть, метки класса (y) заданы DJIA, признаки (X) требуется извлечь из текстов.

Обучающее и тестовое множество строится так: данные до начала 2015 года используются для обучения, данные с 2015 года и позже – для тестировани.

Подсказка:

In [34]:
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [228]:
df0 = pd.read_csv('Combined_News_DJIA.csv')

In [219]:
import pandas.io.date_converters as conv

In [229]:
def always_string(sth): # некоторые строки содержат неполное количество текстов
    if type(sth)!=str: # иногда вместо текстов в TopN содержится float
        return "" # заменяем его пустой строкой
    return sth
def per_col(col):
    return pd.Series({
        "Label": col["Label"],
        "Date": col["Date"],
        "texts": "\n".join([always_string(col["Top{0}".format(c)]) for c in range(1, 26)])
    })
df1 = df0.apply(per_col, axis=1)

In [230]:
train_df = df1[df1['Date'] < '2015-01-01'] # обучающая выборка (x, y, мусор)
test_df = df1[df1['Date'] > '2014-12-31'] # тестовая выборка (x, y, мусор)
y_train = train_df['Label'] # обучающая выборка y
y_test = test_df['Label']# тестовая выборка y

In [227]:
test_df["texts"]

Series([], Name: texts, dtype: object)

In [325]:
def count_vec(voc=None):
    if(voc):
        vectorizer = CountVectorizer(vocabulary=voc) # для (тексты -> x)
        tr = vectorizer.fit_transform(train_df["texts"]) # тексты -> x
        te = vectorizer.fit_transform(test_df["texts"]) # тексты -> x
        return (tr, te)
    else:
        vectorizer = CountVectorizer() # для (тексты -> x)
        tr = vectorizer.fit_transform(train_df["texts"]) # тексты -> x
        voc = vectorizer.get_feature_names()
        vectorizer = CountVectorizer(vocabulary=voc) # для (тексты -> x)
        te = vectorizer.fit_transform(test_df["texts"]) # тексты -> x
        return (tr, te)
train_counts, test_counts = count_vec() # нет словаря
# train_counts, test_counts = count_vec(["Putin", "Norris"]) # есть словарь

In [326]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
# 
x_train = tfidf_transformer.fit_transform(train_counts)
x_test = tfidf_transformer.fit_transform(test_counts)

In [327]:
from sklearn.naive_bayes import MultinomialNB # метод классификации
clf = MultinomialNB().fit(x_train, y_train)
# print(clf)

In [324]:
 # несколько этапов обработки данных можно попытаться ускорить с помощью sklearn Pipeline


In [328]:
predicted = clf.predict(x_test)

In [282]:
from sklearn.linear_model import SGDClassifier
clf2 = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42).fit(x_train, y_train)
predicted_svm = clf2.predict(x_test)

In [284]:
predicted_svm

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1,

Используйте любой известный вам алгоритм классификации текстов для того,  Используйте $tf-idf$ преобразование, сингулярное разложение, нормировку признакого пространства и любые другие техники обработки данных, которые вы считаете нужным. Используйте accuracy и F-measure  для оценки качества классификации. Покажите, как  $tf-idf$ преобразование или сингулярное разложение или любая другая использованная вами техника влияет на качество классификации. 

Если у выбранного вами алгоритма есть гиперпараметры (например, $\alpha$ в преобразовании Лапласа для метода наивного Байеса), покажите, как изменение гиперпараметра влияет на качество классификации. 